# <a id='toc1_'></a>[Projet 7 : Réalisez une analyse de sentiments grâce au Deep Learning](#toc0_)
# <a id='toc2_'></a>[Modèle sur mesure simple](#toc0_)

[Lien OpenClassroom](https://openclassrooms.com/fr/paths/795/projects/1516/1578-mission)

---

**Table of contents**<a id='toc0_'></a>    
- [Projet 7 : Réalisez une analyse de sentiments grâce au Deep Learning](#toc1_)    
- [Modèle sur mesure simple](#toc2_)    
  - [Imports](#toc2_1_)    
  - [Chargement des données](#toc2_2_)    
    - [Chargement du fichier csv](#toc2_2_1_)    
    - [Découpage du jeu de données](#toc2_2_2_)    
  - [Préparation et tests](#toc2_3_)    
    - [Fonctions de préprocessing](#toc2_3_1_)    
    - [ Test des approches de preprocessing](#toc2_3_2_)    
    - [Setup MLFlow](#toc2_3_3_)    
  - [Extraction des features, entrainement du modèle et logging MLFlow](#toc2_4_)    
  - [Évaluation du modèle](#toc2_5_)    
    - [Évaluation sur les données de test](#toc2_5_1_)    
    - [Comparaison avec un modèle naif](#toc2_5_2_)    
    - [Enregistrement du model](#toc2_5_3_)    
  - [Dashboard MLFlow](#toc2_6_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

---
---

## <a id='toc2_1_'></a>[Imports](#toc0_)

In [1]:
import pandas as pd
import mlflow
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    confusion_matrix,
)
from sklearn.dummy import DummyClassifier
import joblib
import os

import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import contractions

try:
    stopwords.words("english")
except LookupError:
    nltk.download("stopwords")
try:
    word_tokenize("test")
except LookupError:
    nltk.download("punkt")
try:
    WordNetLemmatizer().lemmatize("cats")
except LookupError:
    nltk.download("wordnet")
    nltk.download("omw-1.4")

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

pd.set_option("future.no_silent_downcasting", True)

---
---

## <a id='toc2_2_'></a>[Chargement des données](#toc0_)

---

### <a id='toc2_2_1_'></a>[Chargement du fichier csv](#toc0_)

In [2]:
DATA_PATH = "./data.csv"

df = pd.read_csv(DATA_PATH)

df.sample(10)

,tweet,label
1183467,fruit loops with marshmellows lmfao cunt sp...,1
575121,well life sucks! i have to work next sat so no...,0
951487,I recieved an unexpected text! How exciting! ...,1
138012,Planning for a vacation..but no friends to acc...,0
371219,The curious case of benjamen button was so sad,0
1252690,At Ruby Tuesday's with Crystal and Sarah. Fun ...,1
911893,@sharonhayes..where is the song?,1
1414677,And now Uncle Howard's singing... up for 5 Na ...,1
442429,"@Janetita @Danime, That would work, except I d...",0
1148483,@tracecyruss i love the legit wear shirt!,1


---

### <a id='toc2_2_2_'></a>[Découpage du jeu de données](#toc0_)

Cette cellule est dédiée à la division du jeu de données en ensembles d'entraînement, de validation et de test. Elle utilise une approche de double train_test_split pour assurer une répartition stratifiée des labels dans chaque sous-ensemble, garantissant ainsi que la distribution des sentiments est similaire à celle du jeu de données original. Les proportions sont définies pour un ensemble d'entraînement de 70%, un ensemble de validation de 15% et un ensemble de test de 15%. Enfin, les formes des ensembles et la distribution des sentiments au sein de chacun sont affichées pour vérification.

In [3]:
# Définir les features (X) et les labels (y)
X = df["tweet"]
y = df["label"]

# Définir les tailles des ensembles de test et de validation
TEST_SIZE = 0.15
VALIDATION_SIZE = 0.15

# Première division : entraînement vs. (validation + test)
# L'ensemble temporaire contient les données pour la validation et le test
X_train, X_temp, y_train, y_temp = train_test_split(
    X,
    y,
    test_size=(VALIDATION_SIZE + TEST_SIZE),  # Taille combinée pour temp
    random_state=42,  # Pour la reproductibilité
    stratify=y,  # Pour maintenir la distribution des labels
)

# Calculer la proportion de l'ensemble de validation par rapport à l'ensemble temporaire
val_split_ratio = VALIDATION_SIZE / (VALIDATION_SIZE + TEST_SIZE)

# Deuxième division : validation vs. test à partir de l'ensemble temporaire
X_val, X_test, y_val, y_test = train_test_split(
    X_temp,
    y_temp,
    test_size=(1 - val_split_ratio),  # La partie restante sera pour le test
    random_state=42,
    stratify=y_temp,  # Maintenir la distribution des labels dans le sous-ensemble
)

# Afficher les formes des ensembles après la division
print("Division des données terminée :")
print(f"Taille de l'ensemble d'entraînement :   X={X_train.shape}, y={y_train.shape}")
print(f"Taille de l'ensemble de validation : X={X_val.shape}, y={y_val.shape}")
print(f"Taille de l'ensemble de test :       X={X_test.shape}, y={y_test.shape}")

# Afficher la distribution des sentiments dans chaque ensemble
print("\nDistribution des sentiments dans les ensembles :")
print("Entraînement :\n", y_train.value_counts(normalize=True))
print("Validation :\n", y_val.value_counts(normalize=True))
print("Test :\n", y_test.value_counts(normalize=True))

Division des données terminée :
Taille de l'ensemble d'entraînement :   X=(1120000,), y=(1120000,)
Taille de l'ensemble de validation : X=(240000,), y=(240000,)
Taille de l'ensemble de test :       X=(240000,), y=(240000,)

Distribution des sentiments dans les ensembles :
Entraînement :
 label
1    0.5
0    0.5
Name: proportion, dtype: float64
Validation :
 label
0    0.5
1    0.5
Name: proportion, dtype: float64
Test :
 label
1    0.5
0    0.5
Name: proportion, dtype: float64


---
---

## <a id='toc2_3_'></a>[Préparation et tests](#toc0_)

---
### <a id='toc2_3_1_'></a>[Fonctions de préprocessing](#toc0_)

Cette cellule définit trois fonctions de prétraitement du texte, chacune offrant un niveau de nettoyage différent pour la colonne tweet.
- preprocess_full : Effectue un nettoyage complet incluant la conversion en minuscules, l'expansion des contractions, la suppression des URLs, mentions, hashtags, nombres et caractères spéciaux. Elle tokenise ensuite le texte, supprime les stop_words et applique la lemmatisation.

- preprocess_no_stopwords : Similaire à preprocess_full, mais elle conserve les stop_words, se concentrant sur la normalisation des contractions, la suppression des éléments non textuels et la lemmatisation.

- preprocess_none : Cette fonction ne réalise aucun prétraitement, retournant le texte original si c'est une chaîne, ou une chaîne vide sinon.

Ces fonctions sont conçues pour être utilisées ultérieurement afin d'expérimenter l'impact de différentes stratégies de prétraitement sur les performances des modèles.

In [ ]:
def preprocess_full(text):
    """
    Applique toutes les étapes de nettoyage du texte, y compris la suppression des stop words.
    """
    if not isinstance(text, str):
        return ""

    # Convertir le texte en minuscules
    text = text.lower()
    # Étendre les contractions (ex: "don't" -> "do not")
    text = contractions.fix(text)
    # Supprimer les URLs
    text = re.sub(r"http\S+|www\S+", "", text)
    # Supprimer les mentions (@user)
    text = re.sub(r"@\w+", "", text)
    # Supprimer les hashtags (#tag)
    text = re.sub(r"#\w+", "", text)
    # Supprimer les nombres
    text = re.sub(r"\d+", "", text)
    # Supprimer les caractères spéciaux et la ponctuation (ne garder que les lettres et espaces)
    text = re.sub(r"[^a-z\s]", "", text)

    # Tokeniser le texte en mots
    tokens = word_tokenize(text)
    cleaned_tokens = []
    for word in tokens:
        # Supprimer les stop words et les mots d'une seule lettre, puis lemmatiser
        if len(word) > 1 and word not in stop_words:
            lemma = lemmatizer.lemmatize(word)
            cleaned_tokens.append(lemma)
    # Rejoindre les jetons nettoyés en une seule chaîne
    return " ".join(cleaned_tokens)


def preprocess_no_stopwords(text):
    """
    Applique toutes les étapes de nettoyage du texte, SAUF la suppression des stop words.
    """
    if not isinstance(text, str):
        return ""

    text = text.lower()
    text = contractions.fix(text)
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"#\w+", "", text)
    text = re.sub(r"\d+", "", text)
    text = re.sub(r"[^a-z\s]", "", text)

    tokens = word_tokenize(text)
    cleaned_tokens = []
    for word in tokens:
        # Lemmatiser sans supprimer les stop words
        if len(word) > 1:
            lemma = lemmatizer.lemmatize(word)
            cleaned_tokens.append(lemma)
    return " ".join(cleaned_tokens)


def preprocess_none(text):
    """
    N'applique aucun prétraitement, retourne le texte tel quel (ou une chaîne vide si ce n'est pas une chaîne).
    """
    if not isinstance(text, str):
        return ""
    return text

---

### <a id='toc2_3_2_'></a>[ Test des approches de preprocessing](#toc0_)

Cette cellule met en place un processus d'évaluation comparative pour un modèle de régression logistique, en testant l'impact de différentes stratégies de prétraitement du texte.

train_eval_logistic_regression : Cette fonction encapsule le pipeline d'entraînement et d'évaluation. Elle utilise TfidfVectorizer pour convertir le texte prétraité en caractéristiques numériques (fréquences TF-IDF des mots, limitées à 10 000 caractéristiques les plus fréquentes), puis entraîne un modèle LogisticRegression. Elle retourne l'exactitude (accuracy) et le score F1 (métriques de performance du modèle).

Boucle de benchmarking : La cellule itère sur les trois configurations de prétraitement définies précédemment (preprocess_none, preprocess_no_stopwords, preprocess_full). Pour chaque configuration :

- Les ensembles X_train et X_test sont prétraités en utilisant la fonction pp_function correspondante.

- Le modèle de régression logistique est entraîné et évalué avec ces données prétraitées.

- Les résultats (modèle utilisé, type de prétraitement, accuracy, F1-Score) sont stockés dans la liste results_log.

L'objectif est d'identifier quelle stratégie de prétraitement optimise les performances du modèle de régression logistique pour la tâche d'analyse de sentiment.

In [5]:
def train_eval_logistic_regression(
    X_train_processed, y_train, X_test_processed, y_test
):
    """
    Entraîne et évalue un modèle de régression logistique avec TF-IDF.

    Args:
        X_train_processed (list): Textes d'entraînement prétraités.
        y_train (pd.Series): Labels d'entraînement.
        X_test_processed (list): Textes de test prétraités.
        y_test (pd.Series): Labels de test.

    Returns:
        tuple: (accuracy, f1_score)
    """
    print("Entraînement de la Régression Logistique...")
    # Initialiser et entraîner un vectorisateur TF-IDF sur les données d'entraînement
    vectorizer = TfidfVectorizer(max_features=10000)
    X_train_tfidf = vectorizer.fit_transform(X_train_processed)
    # Transformer les données de test avec le vectorisateur entraîné
    X_test_tfidf = vectorizer.transform(X_test_processed)

    # Initialiser et entraîner le modèle de régression logistique
    model = LogisticRegression(max_iter=500)
    model.fit(X_train_tfidf, y_train)
    # Faire des prédictions sur l'ensemble de test
    predictions = model.predict(X_test_tfidf)
    # Calculer l'exactitude (accuracy)
    acc = accuracy_score(y_test, predictions)
    # Calculer le score F1 (pour les classes binaires)
    f1 = f1_score(y_test, predictions, average="binary")
    return acc, f1


# Définir les différentes configurations de prétraitement à tester
preprocessing_configs = {
    "Sans Prétraitement": preprocess_none,
    "Tout Sauf stop words": preprocess_no_stopwords,
    "Tout Prétraitement": preprocess_full,
}

results_log = []  # Liste pour stocker les résultats

# Boucler sur chaque configuration de prétraitement
for pp_name, pp_function in preprocessing_configs.items():
    print(f"\n--- BENCHMARKING AVEC PRÉTRAITEMENT : {pp_name} ---")

    # Appliquer la fonction de prétraitement aux ensembles d'entraînement et de test
    current_X_train = [pp_function(text) for text in X_train]
    current_X_test = [pp_function(text) for text in X_test]

    # Utiliser les labels non modifiés
    current_y_train = y_train
    current_y_test = y_test

    # --- Régression Logistique ---
    # Entraîner et évaluer le modèle avec les données prétraitées
    acc, f1 = train_eval_logistic_regression(
        current_X_train, current_y_train, current_X_test, current_y_test
    )
    # Enregistrer les résultats
    results_log.append(
        {
            "Modèle": "LogisticRegression+TFIDF",
            "Prétraitement": pp_name,
            "Accuracy": acc,
            "F1-Score": f1,
        }
    )
    print(f"Régression Logistique - Accuracy: {acc:.4f}, F1-Score: {f1:.4f}")


--- BENCHMARKING AVEC PRÉTRAITEMENT : Sans Prétraitement ---
Entraînement de la Régression Logistique...
Régression Logistique - Accuracy: 0.7935, F1-Score: 0.7961

--- BENCHMARKING AVEC PRÉTRAITEMENT : Tout Sauf Mots Vides ---
Entraînement de la Régression Logistique...
Régression Logistique - Accuracy: 0.7921, F1-Score: 0.7946

--- BENCHMARKING AVEC PRÉTRAITEMENT : Tout Prétraitement ---
Entraînement de la Régression Logistique...
Régression Logistique - Accuracy: 0.7684, F1-Score: 0.7735


---

### <a id='toc2_3_3_'></a>[Setup MLFlow](#toc0_)

Cette cellule initialise et configure l'environnement MLflow pour le suivi des expériences. Elle définit un nom d'expérience (EXPERIMENT_NAME) et tente de la définir comme l'expérience MLflow active. Si l'expérience n'existe pas, elle est créée. L'objectif est de centraliser et de versionner les résultats et les artefacts des différents runs du modèle. De plus, elle définit une constante pour le nom de fichier qui sera utilisé pour sauvegarder le vectoriseur TF-IDF, un composant clé du pipeline de traitement du texte.

In [6]:
# Nom de l'expérience MLflow
EXPERIMENT_NAME = "Tweet Sentiment Analysis - Simple Models"
# Définir l'expérience MLflow courante
mlflow.set_experiment(EXPERIMENT_NAME)

# Récupérer les détails de l'expérience courante (ou en créer une si elle n'existe pas)
try:
    experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
    if experiment is None:
        experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
        print(f"Nouvelle expérience créée avec ID : {experiment_id}")
    else:
        experiment_id = experiment.experiment_id
        print(
            f"Utilisation de l'expérience existante '{EXPERIMENT_NAME}' avec ID : {experiment_id}"
        )
except Exception as e:
    print(f"Erreur lors de la configuration de l'expérience MLflow : {e}")
    raise

# Nom de fichier pour sauvegarder le vectoriseur TF-IDF
VECTORIZER_FILENAME = "tfidf_vectorizer_simple.joblib"

Utilisation de l'expérience existante 'Tweet Sentiment Analysis - Simple Models' avec ID : 897408299388468996


---
---

## <a id='toc2_4_'></a>[Extraction des features, entrainement du modèle et logging MLFlow](#toc0_)

Cette cellule encapsule l'intégralité du processus d'entraînement et d'évaluation d'un modèle de régression logistique pour l'analyse de sentiment, tout en utilisant MLflow pour le suivi et la gestion de l'expérience.

Initialisation MLflow : Une nouvelle exécution MLflow est démarrée sous le nom spécifié.

Paramètres : Les hyperparamètres clés du vectoriseur TF-IDF (comme max_features et ngram_range) et du modèle de régression logistique (comme C, solver, max_iter, class_weight) sont définis et enregistrés en tant que paramètres MLflow.

Extraction de Caractéristiques (TF-IDF) : Un TfidfVectorizer est entraîné sur l'ensemble d'entraînement et utilisé pour transformer les ensembles d'entraînement, de validation et de test en représentations numériques. Le vectoriseur entraîné est ensuite sauvegardé localement et enregistré en tant qu'artefact MLflow pour une réutilisation future.

Entraînement du Modèle : Le modèle LogisticRegression est initialisé avec les paramètres définis et entraîné sur les données d'entraînement transformées.

Évaluation sur l'Ensemble de Validation : Le modèle est évalué sur l'ensemble de validation en utilisant des métriques clés comme l'accuracy, la précision, le rappel et le score F1. Toutes ces métriques sont enregistrées dans MLflow.

Enregistrement du Modèle et Rapports : Le modèle entraîné est enregistré en tant qu'artefact MLflow. De plus, un rapport de classification détaillé est généré et également enregistré en tant qu'artefact pour une analyse approfondie.

L'objectif principal de cette cellule est de documenter de manière exhaustive chaque aspect de cette exécution modèle au sein de MLflow, permettant une traçabilité, une reproductibilité et une comparaison facile avec d'autres expériences.

In [7]:
# Nom de l'exécution MLflow pour ce modèle
run_name = "LogisticRegression_TFIDF"
print(f"\nDébut de l'exécution MLflow : {run_name}")

# Démarrer une nouvelle exécution MLflow
with mlflow.start_run(run_name=run_name) as run:
    run_id = run.info.run_id
    print(f"ID de l'exécution MLflow : {run_id}")

    # --- Paramètres ---
    # Paramètres du vectoriseur TF-IDF
    tfidf_max_features = 10000  # Nombre maximum de caractéristiques à considérer
    tfidf_ngram_range = (1, 3)  # Inclure les unigrammes, bigrammes et trigrammes

    # Paramètres de la Régression Logistique
    lr_C = 0.5  # Inverse de la force de régularisation (plus C est petit, plus la régularisation est forte)
    lr_solver = "saga"  # Algorithme d'optimisation
    lr_max_iter = 500  # Nombre maximum d'itérations pour la convergence
    lr_class_weight = "balanced"  # Ajuster automatiquement les poids pour gérer le déséquilibre de classe

    # Enregistrer les paramètres dans MLflow
    print("Enregistrement des paramètres...")
    mlflow.log_param("type_vectoriseur", "TF-IDF")
    mlflow.log_param("tfidf_max_features", tfidf_max_features)
    mlflow.log_param(
        "tfidf_ngram_range", str(tfidf_ngram_range)
    )  # Tuple converti en string
    mlflow.log_param("type_modele", "RegressionLogistique")
    mlflow.log_param("lr_C", lr_C)
    mlflow.log_param("lr_solver", lr_solver)
    mlflow.log_param("lr_max_iter", lr_max_iter)
    mlflow.log_param("lr_class_weight", lr_class_weight)

    # --- Extraction de caractéristiques (TF-IDF) ---
    print("Entraînement du vectoriseur TF-IDF...")
    vectorizer = TfidfVectorizer(
        max_features=tfidf_max_features, ngram_range=tfidf_ngram_range
    )
    # Entraîner le vectoriseur sur les données d'entraînement et transformer
    X_train_tfidf = vectorizer.fit_transform(X_train)
    print(
        f"TF-IDF - Forme des données d'entraînement transformées : {X_train_tfidf.shape}"
    )

    # Transformer les ensembles de validation et de test avec le vectoriseur entraîné
    X_val_tfidf = vectorizer.transform(X_val)
    X_test_tfidf = vectorizer.transform(X_test)
    print(
        f"TF-IDF - Forme des données de validation transformées : {X_val_tfidf.shape}"
    )
    print(f"TF-IDF - Forme des données de test transformées : {X_test_tfidf.shape}")

    # Sauvegarder le vectoriseur entraîné localement
    joblib.dump(vectorizer, VECTORIZER_FILENAME)
    print(f"Vectoriseur sauvegardé localement dans {VECTORIZER_FILENAME}")

    # Enregistrer le vectoriseur en tant qu'artefact MLflow
    mlflow.log_artifact(VECTORIZER_FILENAME, artifact_path="vectoriseur")
    print("Vectoriseur enregistré comme artefact MLflow.")

    # Supprimer le fichier local après l'enregistrement (optionnel)
    if os.path.exists(VECTORIZER_FILENAME):
        os.remove(VECTORIZER_FILENAME)

    # --- Entraînement du Modèle ---
    print("Entraînement du modèle de Régression Logistique...")
    model = LogisticRegression(
        C=lr_C,
        solver=lr_solver,
        max_iter=lr_max_iter,
        class_weight=lr_class_weight,
        random_state=42,  # Pour la reproductibilité
    )
    model.fit(X_train_tfidf, y_train)
    print("Entraînement du modèle terminé.")

    # --- Évaluation sur l'ensemble de Validation ---
    print("Évaluation sur l'ensemble de validation...")
    y_val_pred = model.predict(X_val_tfidf)
    # Probabilité de la classe positive (utile pour les courbes ROC, etc.)
    y_val_pred_proba = model.predict_proba(X_val_tfidf)[:, 1]

    # Calculer les métriques d'évaluation
    val_accuracy = accuracy_score(y_val, y_val_pred)
    val_precision = precision_score(y_val, y_val_pred, zero_division=0)
    val_recall = recall_score(y_val, y_val_pred, zero_division=0)
    val_f1 = f1_score(y_val, y_val_pred, zero_division=0)

    # Enregistrer les métriques de validation dans MLflow
    print("Enregistrement des métriques de validation...")
    mlflow.log_metric("val_accuracy", val_accuracy)
    mlflow.log_metric("val_precision", val_precision)
    mlflow.log_metric("val_recall", val_recall)
    mlflow.log_metric("val_f1", val_f1)

    print(f"Validation Accuracy : {val_accuracy:.4f}")
    print(f"Validation Précision : {val_precision:.4f}")
    print(f"Validation Rappel : {val_recall:.4f}")
    print(f"Validation F1-Score : {val_f1:.4f}")

    # --- Enregistrer le Modèle ---
    print("Enregistrement du modèle entraîné...")
    mlflow.sklearn.log_model(model, artifact_path="modele-regression-logistique")
    print("Modèle enregistré avec succès.")

    # Générer et enregistrer le rapport de classification détaillé
    val_report = classification_report(y_val, y_val_pred, output_dict=True)
    mlflow.log_dict(val_report, "rapport_classification_validation.json")

print(f"\nExécution MLflow {run_id} terminée.")


Début de l'exécution MLflow : LogisticRegression_TFIDF
ID de l'exécution MLflow : d43f58cea498442dbbb0035cdc96cc82
Enregistrement des paramètres...
Entraînement du vectoriseur TF-IDF...
TF-IDF - Forme des données d'entraînement transformées : (1120000, 10000)
TF-IDF - Forme des données de validation transformées : (240000, 10000)
TF-IDF - Forme des données de test transformées : (240000, 10000)
Vectoriseur sauvegardé localement dans tfidf_vectorizer_simple.joblib
Vectoriseur enregistré comme artefact MLflow.
Entraînement du modèle de Régression Logistique...
Entraînement du modèle terminé.
Évaluation sur l'ensemble de validation...
Enregistrement des métriques de validation...
Validation Accuracy : 0.8051
Validation Précision : 0.7981
Validation Rappel : 0.8169
Validation F1-Score : 0.8074
Enregistrement du modèle entraîné...


2025/06/17 19:22:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Modèle enregistré avec succès.

Exécution MLflow d43f58cea498442dbbb0035cdc96cc82 terminée.


---
---

## <a id='toc2_5_'></a>[Évaluation du modèle](#toc0_)

---

### <a id='toc2_5_1_'></a>[Évaluation sur les données de test](#toc0_)

Cette cellule se concentre sur l'évaluation finale du modèle entraîné et enregistré via MLflow, en utilisant l'ensemble de test.

Chargement du Modèle : Elle tente de charger le modèle de régression logistique à partir de son URI MLflow (logged_model_uri) en utilisant la fonction mlflow.sklearn.load_model.

Prédictions et Évaluation : Une fois le modèle chargé, elle réalise des prédictions sur l'ensemble de test (X_test_tfidf) et calcule les métriques de performance standard (accuracy, precision, recall, F1-score).

Affichage des Résultats : Les métriques de performance, le rapport de classification et la matrice de confusion pour l'ensemble de test sont imprimés, offrant une vue complète de la capacité généralisation du modèle.

Journalisation dans MLflow : Enfin, les métriques de test et le rapport de classification détaillé sont journalisés rétroactivement dans la même exécution MLflow qui a enregistré le modèle. Ceci est crucial pour avoir toutes les informations de performance (validation et test) associées à une seule exécution dans l'interface utilisateur de MLflow, facilitant ainsi la comparaison et l'analyse.

Cette étape est essentielle pour obtenir une estimation fiable des performances du modèle sur des données non vues, et pour documenter ces résultats dans MLflow.

In [8]:
# Construire l'URI pour charger le modèle enregistré dans l'exécution MLflow précédente
logged_model_uri = f"runs:/{run_id}/modele-regression-logistique"

# Tenter de charger le modèle et d'évaluer ses performances sur l'ensemble de test
try:
    # Charger le modèle à partir de l'URI MLflow
    loaded_model = mlflow.sklearn.load_model(logged_model_uri)
    print(f"Modèle chargé avec succès depuis : {logged_model_uri}")

    # Faire des prédictions sur l'ensemble de test (en utilisant la transformation TF-IDF existante)
    y_test_pred = loaded_model.predict(X_test_tfidf)
    # Obtenir les probabilités de la classe positive
    y_test_pred_proba = loaded_model.predict_proba(X_test_tfidf)[:, 1]

    # Calculer les métriques de performance sur l'ensemble de test
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_precision = precision_score(y_test, y_test_pred, zero_division=0)
    test_recall = recall_score(y_test, y_test_pred, zero_division=0)
    test_f1 = f1_score(y_test, y_test_pred, zero_division=0)

    print("\nPerformances sur l'ensemble de test :")
    print(f"Test Accuracy : {test_accuracy:.4f}")
    print(f"Test Précision : {test_precision:.4f}")
    print(f"Test Rappel : {test_recall:.4f}")
    print(f"Test F1-Score : {test_f1:.4f}")

    print("\nRapport de classification (ensemble de test) :")
    print(classification_report(y_test, y_test_pred))

    print("\nMatrice de confusion (ensemble de test) :")
    print(confusion_matrix(y_test, y_test_pred))

    # --- Optionnel : Enregistrer les métriques de test dans la même exécution MLflow ---
    # Pour cela, il faut utiliser le client MLflow car l'exécution est déjà terminée.
    client = mlflow.tracking.MlflowClient()
    client.log_metric(run_id, "test_accuracy", test_accuracy)
    client.log_metric(run_id, "test_precision", test_precision)
    client.log_metric(run_id, "test_recall", test_recall)
    client.log_metric(run_id, "test_f1", test_f1)
    print("\nMétriques de test enregistrées dans l'exécution MLflow.")

    # Enregistrer également le rapport de classification de test
    test_report_dict = classification_report(y_test, y_test_pred, output_dict=True)
    client.log_dict(run_id, test_report_dict, "rapport_classification_test.json")


except Exception as e:
    print(
        f"Erreur lors du chargement du modèle ou de l'évaluation sur l'ensemble de test : {e}"
    )
    print(
        "Assurez-vous que l'ID de l'exécution est correct et que le modèle a été correctement enregistré."
    )

Modèle chargé avec succès depuis : runs:/d43f58cea498442dbbb0035cdc96cc82/modele-regression-logistique

Performances sur l'ensemble de test :
Test Accuracy : 0.8041
Test Précision : 0.7972
Test Rappel : 0.8156
Test F1-Score : 0.8063

Rapport de classification (ensemble de test) :
              precision    recall  f1-score   support

           0       0.81      0.79      0.80    120000
           1       0.80      0.82      0.81    120000

    accuracy                           0.80    240000
   macro avg       0.80      0.80      0.80    240000
weighted avg       0.80      0.80      0.80    240000


Matrice de confusion (ensemble de test) :
[[95110 24890]
 [22128 97872]]

Métriques de test enregistrées dans l'exécution MLflow.


---

### <a id='toc2_5_2_'></a>[Comparaison avec un modèle naif](#toc0_)

Cette cellule établit une "ligne de base" (baseline) pour l'évaluation des performances du modèle. Elle utilise un DummyClassifier avec la stratégie "most_frequent", ce qui signifie qu'il prédit toujours la classe majoritaire de l'ensemble d'entraînement. Cela permet de déterminer la performance minimale qu'un modèle devrait dépasser pour être considéré comme utile. Le classifieur dummy est entraîné sur les données TF-IDF d'entraînement, puis ses performances (accuracy, precision, recall, F1-score) sont calculées et affichées sur l'ensemble de test. Enfin, ces métriques de base sont comparées à celles du modèle de régression logistique entraîné, afin de quantifier le gain apporté par le modèle plus sophistiqué.

In [9]:
# Initialiser un classifieur "dummy" (naïf) utilisant la stratégie de la classe la plus fréquente
dummy_clf = DummyClassifier(strategy="most_frequent")
# Entraîner le classifieur dummy (nécessaire même si la stratégie est simple, pour identifier la classe majoritaire)
dummy_clf.fit(X_train_tfidf, y_train)

# Faire des prédictions sur l'ensemble de test avec le classifieur dummy
y_test_pred_dummy = dummy_clf.predict(X_test_tfidf)

# Calculer les métriques de performance pour la ligne de base
dummy_accuracy = accuracy_score(y_test, y_test_pred_dummy)
dummy_precision = precision_score(y_test, y_test_pred_dummy, zero_division=0)
dummy_recall = recall_score(y_test, y_test_pred_dummy, zero_division=0)
dummy_f1 = f1_score(y_test, y_test_pred_dummy, zero_division=0)

print("Performances de la ligne de base naïve (ensemble de test) :")
print(f"Accuracy de base : {dummy_accuracy:.4f}")
print(f"Précision de base : {dummy_precision:.4f}")
print(f"Rappel de base : {dummy_recall:.4f}")
print(f"F1-Score de base : {dummy_f1:.4f}")

print("\nComparaison :")
print(
    f"Accuracy du modèle sur le test : {test_accuracy:.4f} vs Ligne de base : {dummy_accuracy:.4f}"
)
print(
    f"F1-Score du modèle sur le test : {test_f1:.4f} vs Ligne de base : {dummy_f1:.4f}"
)

Performances de la ligne de base naïve (ensemble de test) :
Accuracy de base : 0.5000
Précision de base : 0.0000
Rappel de base : 0.0000
F1-Score de base : 0.0000

Comparaison :
Accuracy du modèle sur le test : 0.8041 vs Ligne de base : 0.5000
F1-Score du modèle sur le test : 0.8063 vs Ligne de base : 0.0000


---

### <a id='toc2_5_3_'></a>[Enregistrement du model](#toc0_)

Cette cellule enregistre le modèle entraîné et journalisé précédemment dans le registre de modèles MLflow. En fournissant l'URI du modèle à partir de l'exécution MLflow (logged_model_uri) et un nom ("MODEL_SIMPLE"), le modèle est catalogué et versionné au sein du registre. Cela permet une gestion centralisée des versions du modèle, facilitant son déploiement et sa réutilisation dans des environnements de production. Les informations clés du modèle enregistré (nom, version, étape actuelle) sont ensuite affichées.

In [10]:
# Enregistrer le modèle entraîné dans le registre de modèles MLflow
registered_model_info = mlflow.register_model(
    model_uri=logged_model_uri,  # URI du modèle enregistré dans l'exécution précédente
    name="MODEL_SIMPLE",  # Nom sous lequel le modèle sera enregistré
)

print("Modèle enregistré avec succès :")
print(f"- Nom : {registered_model_info.name}")
print(f"- Version : {registered_model_info.version}")
print(f"- Étape : {registered_model_info.current_stage}")

Modèle enregistré avec succès :
- Nom : MODEL_SIMPLE
- Version : 5
- Étape : None


Registered model 'MODEL_SIMPLE' already exists. Creating a new version of this model...
Created version '5' of model 'MODEL_SIMPLE'.


---
---

## <a id='toc2_6_'></a>[Dashboard MLFlow](#toc0_)

In [11]:
! mlflow server --host 127.0.0.1 --port 8080

^C


![Overview](./mlflow_screenshot/simple/Overview.png)

![Metrics](./mlflow_screenshot/simple/Metrics.png)

![Compare Runs](./mlflow_screenshot/simple/Compare_runs.png)